Let's solve a rubik's cube.

In [221]:
import numpy as np

class Cube:
    def __init__(self):
        self.cube = np.reshape(np.outer(np.outer(np.arange(1,6),np.arange(1,6)),np.arange(1,6)),(5,5,5))

    @property
    def f(self):
        self.cube[-1] = np.flip(self.cube[-1].T,1)
        return self

In [231]:
c = Cube()
c.f.f.cube[-1]

array([[125, 100,  75,  50,  25],
       [100,  80,  60,  40,  20],
       [ 75,  60,  45,  30,  15],
       [ 50,  40,  30,  20,  10],
       [ 25,  20,  15,  10,   5]])

In [182]:
# front clockwise
np.flip(h[-1].T,1)

array([[ 25,  20,  15,  10,   5],
       [ 50,  40,  30,  20,  10],
       [ 75,  60,  45,  30,  15],
       [100,  80,  60,  40,  20],
       [125, 100,  75,  50,  25]])

In [183]:
# front counter clockwise
np.flip(h[-1].T,0)

array([[ 25,  50,  75, 100, 125],
       [ 20,  40,  60,  80, 100],
       [ 15,  30,  45,  60,  75],
       [ 10,  20,  30,  40,  50],
       [  5,  10,  15,  20,  25]])

In [179]:
# top clockwise
np.flip(h[:,:1].T,-1)

array([[[ 5,  4,  3,  2,  1]],

       [[10,  8,  6,  4,  2]],

       [[15, 12,  9,  6,  3]],

       [[20, 16, 12,  8,  4]],

       [[25, 20, 15, 10,  5]]])

In [180]:
# top counter clockwise
np.flip(h[:,:1].T,0)

array([[[ 5, 10, 15, 20, 25]],

       [[ 4,  8, 12, 16, 20]],

       [[ 3,  6,  9, 12, 15]],

       [[ 2,  4,  6,  8, 10]],

       [[ 1,  2,  3,  4,  5]]])

In [197]:
# side clockwise
np.rot90(h[:,:,:1],axes=(0,1))

array([[[ 5],
        [10],
        [15],
        [20],
        [25]],

       [[ 4],
        [ 8],
        [12],
        [16],
        [20]],

       [[ 3],
        [ 6],
        [ 9],
        [12],
        [15]],

       [[ 2],
        [ 4],
        [ 6],
        [ 8],
        [10]],

       [[ 1],
        [ 2],
        [ 3],
        [ 4],
        [ 5]]])

In [198]:
# side counter clockwise
np.rot90(h[:,:,:1],axes=(1,0))

array([[[ 5],
        [ 4],
        [ 3],
        [ 2],
        [ 1]],

       [[10],
        [ 8],
        [ 6],
        [ 4],
        [ 2]],

       [[15],
        [12],
        [ 9],
        [ 6],
        [ 3]],

       [[20],
        [16],
        [12],
        [ 8],
        [ 4]],

       [[25],
        [20],
        [15],
        [10],
        [ 5]]])

In [201]:
h[:,:,1:2] # l
h[:,:,2:3] # m
h[:,:,3:4] # r
h[:,1:2]   # u
h[:,2:3]   # e
h[:,3:4]   # d
h[1]       # b
h[2]       # s
h[3]       # f

array([[[ 2],
        [ 4],
        [ 6],
        [ 8],
        [10]],

       [[ 4],
        [ 8],
        [12],
        [16],
        [20]],

       [[ 6],
        [12],
        [18],
        [24],
        [30]],

       [[ 8],
        [16],
        [24],
        [32],
        [40]],

       [[10],
        [20],
        [30],
        [40],
        [50]]])